In [ ]:
import torch
from torch_geometric.data import Data



In [ ]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary, Edge_list, Edge_features = get_files()


In [ ]:
edge_index = torch.tensor(Edge_list, dtype=torch.long)
edge_features = torch.tensor(Edge_features, dtype=torch.float)

print(edge_index.shape)
print(edge_features.shape)

data = Data(edge_index=edge_index.t().contiguous(), edge_features=edge_features)



In [ ]:
#Okay, so your graph is directed until you invert fuckin everythin 
data.is_directed()

In [ ]:
data.edge_features
#Fix this shit my dude